# Introduction to Secure Multiparty Computation (SMPC)

This is the first notebook that will explain the basics of multiparty computation. In essence multiparty computation tries to outsource computations to different parties. Now, this per se is easy right? If I want a certain calculation to be performed in a remote computer I just send the data, the function to be calculated and that's it. The problem arises when one wants to outsource a computation without revealing the data, just the function. Secure Multiparty Computation (SMPC) is a set of techniques that allows us to compute a function without explicitely knowing the inputs.


## Yao's Millionaire problem

The typical application example of SMPC is the Yao's millionaire problem: Lets assume there are two millionaires, named Alice and Bob. Alice and Bob what to know who is richer without revealing how much money either one of them have. This is what is known as the millionaires problem, originally introduced by [Andrew Yao](https://en.wikipedia.org/wiki/Andrew_Yao). In essence they just want to know a bit of information, who's richer but not by which ammount. 

Now, let's assume we are an observer and we know Alice's and Bob net worth in the bank account

In [1]:
Alice_Wealth = 10
Bob_Wealth = 5
n = 20 #values to compute

Let's begin the protocol with Alice generating private and public keys for RSA protocol.

In [2]:
from crypt import RSAKeyGenerator
from random import seed
#seed(3)

bits = 16
PublicKey, PrivateKey = RSAKeyGenerator(bits)

assert(PublicKey[0] == PrivateKey[0])
N = PublicKey[0]
e = PublicKey[1]
d = PrivateKey[1]

print(f"(N, e, d) = {N, e, d}")

(N, e, d) = (2496236737, 194181079, 80892601)


Bob takes the values of the public key generated by Alice and chooses a random number U in N

In [3]:
from crypt import fastPowering
from random import randrange
from crypt import RSAEncrypt

U = randrange(2, 1<<bits)
C = RSAEncrypt(U, PublicKey) #encrypted U using e public key

print(f"Bob has chosen U={U} at random, whose encryption is C={C}")

Bob has chosen U=31756 at random, whose encryption is C=1940055869


Now Bob has to send to Alice a value Y1=C-BobWealth-1

In [4]:
y1 = C-Bob_Wealth
print(f"y1 = {y1}")

y1 = 1940055864


Alice calculates the RSA encryption of Y1, Y2... Yn.

In [6]:
from crypt import RSAEncrypt

c = []
y = []

for x in range(n):
    c.append(y1+x-1)
    y.append(RSAEncrypt(c[-1], PublicKey))
    
    print(f"C-BobWealth+{x} = {c[-1]}, y{x}={y[-1]}")

C-BobWealth+0 = 1940055863, y0=678517380
C-BobWealth+1 = 1940055864, y1=2492264854
C-BobWealth+2 = 1940055865, y2=2441905788
C-BobWealth+3 = 1940055866, y3=1612337013
C-BobWealth+4 = 1940055867, y4=70191204
C-BobWealth+5 = 1940055868, y5=392864053
C-BobWealth+6 = 1940055869, y6=2386450628
C-BobWealth+7 = 1940055870, y7=1130473205
C-BobWealth+8 = 1940055871, y8=629111310
C-BobWealth+9 = 1940055872, y9=2441768318
C-BobWealth+10 = 1940055873, y10=847883477
C-BobWealth+11 = 1940055874, y11=735990958
C-BobWealth+12 = 1940055875, y12=252190394
C-BobWealth+13 = 1940055876, y13=21442690
C-BobWealth+14 = 1940055877, y14=1629785012
C-BobWealth+15 = 1940055878, y15=445056373
C-BobWealth+16 = 1940055879, y16=2325127357
C-BobWealth+17 = 1940055880, y17=2052048843
C-BobWealth+18 = 1940055881, y18=1551463887
C-BobWealth+19 = 1940055882, y19=2266904035


Alice has to choose a random prime p in the range bits/2.

In [7]:
from crypt import RandomPrime
p = RandomPrime(bits//2, 40)

print(f"Alice has chosen p={p} secretly")

Alice has chosen p=227 secretly


Now with this secret value p prime she calculates yn (mod p)

In [8]:
z = []

for i, yx in enumerate(y):
    z.append(yx%p)
    print(f"z{i}={z[-1]}")

z0=79
z1=74
z2=50
z3=51
z4=80
z5=147
z6=82
z7=39
z8=105
z9=142
z10=114
z11=208
z12=204
z13=43
z14=149
z15=173
z16=180
z17=169
z18=153
z19=88


Now she wants to send zx to Bob but adding 1 from the value she has to the end:

In [9]:
zp = []

for i, y in enumerate(z):
    if i<Alice_Wealth:
        zp.append(y)
    else:
        zp.append(y+1)
    print(f"zp{i}={zp[-1]}")

zp0=79
zp1=74
zp2=50
zp3=51
zp4=80
zp5=147
zp6=82
zp7=39
zp8=105
zp9=142
zp10=115
zp11=209
zp12=205
zp13=44
zp14=150
zp15=174
zp16=181
zp17=170
zp18=154
zp19=89


In [10]:
print(f"Nottice that since worth of Alice is {Alice_Wealth} their indexes start to change from this index onwards")

Nottice that since worth of Alice is 10 their indexes start to change from this index onwards


Alice now sends the values of $zp$ to Bob and the prime number p. Now Bob looks at the index of his worth value and calculates G = U (mod p)

In [11]:
G = U%p
lookup_value = zp[Bob_Wealth]
print(f"z{Bob_Wealth}={lookup_value}")
print(f"G={G}")

z5=147
G=203


In [12]:
if lookup_value == G:
    print("Alice worth is equal or greater to Bob's")
else:
    print("Alice worth is smaller to Bob's")

Alice worth is smaller to Bob's
